In [87]:
import sys
import json

params = {'general':{'period':'am'}}
default = {'training_folder': '../..', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../..', 'params': {'general': {'period': 'am'}}}


In [88]:
def set_index(df):
    if 'index' in df.columns:
        df = df.set_index('index')
    else:
        df.index.name='index'
    return df

In [89]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel


In [90]:
period = argv['params']['general']['period']
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)
training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'

if on_lambda:
    scenario_folder = training_folder +r'/inputs/'
    output_folder = training_folder + '/outputs/'
    model_folder = training_folder + '/model/'
else:
    scenario_folder = training_folder + '/scenarios/' + period + '/inputs/'
    output_folder = training_folder + '/scenarios/' + period + '/outputs/'
    model_folder = training_folder + '/scenarios/' + period + '/model/'
print('input folder: ', input_folder)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folder)
print('model folder : ', model_folder)


pt_folder = scenario_folder + 'pt/'
road_folder = scenario_folder + 'road/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

On Lambda :  False
input folder:  ../../inputs/
output folder:  ../../scenarios/am/outputs/
scen folder :  ../../scenarios/am/inputs/
model folder :  ../../scenarios/am/model/


In [91]:
var, ancestry = excel.read_var(file=input_folder+'/parameters.xlsx', scenario=period, return_ancestry=True)


# inputs

In [92]:
sm = stepmodel.read_zippedpickles(model_folder + 'network')

road_nodes: 100%|███████████████████████████████| 10/10 [00:00<00:00, 21.76it/s]


volumes

In [93]:
filepath = excel.get_filepath(input_folder + '{s}/volumes/volumes.csv', ancestry=ancestry, log=False)
volumes = pd.read_csv(filepath)
volumes = set_index(volumes)

In [94]:
period_dict={'am':'AM','pm':'PM','ip':'IP','op':'OP'}
volumes = volumes[volumes['time_period'] == period_dict[period]]

In [95]:
volumes['origin'] = 'zone_' + volumes['origin'].astype(str)
volumes['destination'] = 'zone_' + volumes['destination'].astype(str)

congested road time

In [96]:
road_times = pd.read_csv(road_folder+'road_times.csv')
road_times = set_index(road_times)
road_times['old_index'] = road_times.index
road_times.index = 'rlink_' + road_times.index.astype(str)

# apply traffic

In [97]:
road_times['jam_factor'] = sm.road_links['time']/road_times['time']

In [98]:
jam_factor = road_times.set_index('old_index')['jam_factor'].to_dict()
sm.links['jam_factor'] = sm.links['road_link_list'].apply(lambda ls: sum([*map(jam_factor.get, ls)]))
sm.links[['time','jam_factor']]

,time,jam_factor
links_524_1,3.361757,0.914359
links_524_2,14.114571,0.906643
links_524_3,5.618655,0.909467
links_524_4,11.277692,0.913957
links_524_5,5.212693,0.952833
...,...,...
links_200_27,52.989173,0.945482
links_200_28,44.131096,0.936969
links_200_29,130.797420,0.925762
links_200_30,1078.045188,0.911252


In [99]:
ref = sm.copy()

In [100]:
sm.links['time'] = sm.links['time'] / sm.links['jam_factor'] 

# pathfinder

In [101]:
od_set = set(zip(volumes['origin'], volumes['destination']))
#od_set ={('zone_565', 'zone_413')}


In [102]:
sm.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=False,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=False,
    path_analysis=False,
    od_set=od_set,
    engine='b', # b is faster
)
sm.pt_los['time'] = sm.pt_los['gtime']

start publicpathfinder
path_analysis


export

In [103]:
sm.pt_los[['origin','destination','time']].to_csv(scenario_folder +'pt_skim.csv')

# ref pathfinder

In [104]:
ref.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=False,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=False,
    path_analysis=False,
    od_set=od_set,
    engine='b', # b is faster
)
ref.pt_los['time'] = ref.pt_los['gtime']

start publicpathfinder
path_analysis


In [105]:
ref.pt_los[['origin','destination','time']].to_csv(scenario_folder +'ref_pt_skim.csv')

In [106]:
#sm.to_zippedpickles(model_folder +'los', omitted_attributes=['publicpathfinder'])

# Car pathfinder

In [124]:

car = sm.copy()
car.road_links['dup'] = car.road_links['a'] + car.road_links['b']
car.road_links = car.road_links.drop_duplicates('dup').drop(columns=['dup'])

car.preparation_ntlegs(
    short_leg_speed=15,
    long_leg_speed=15,
    threshold=5000,
    zone_to_transit=False,
    zone_to_road=True,
    n_ntlegs=1,
    max_ntleg_length=10000000000,
)
car.step_road_pathfinder(method='aon', access_time='time',od_set=od_set)


self.volumes does not exist. od generated with self.zones


In [125]:
car.car_los[['origin','destination','time']].to_csv(scenario_folder +'car_skim.csv')

then reference with time = road_times

In [126]:
car = ref.copy()

car.road_links['time']  = road_times['time']

car.road_links['dup'] = car.road_links['a'] + car.road_links['b']
car.road_links = car.road_links.drop_duplicates('dup').drop(columns=['dup'])

car.preparation_ntlegs(
    short_leg_speed=15,
    long_leg_speed=15,
    threshold=5000,
    zone_to_transit=False,
    zone_to_road=True,
    n_ntlegs=1,
    max_ntleg_length=10000000000,
)
car.step_road_pathfinder(method='aon', access_time='time',od_set=od_set)


self.volumes does not exist. od generated with self.zones


In [127]:
car.car_los[['origin','destination','time']].to_csv(scenario_folder +'ref_car_skim.csv')